<a href="https://colab.research.google.com/github/sanyhmahadi/Fake-News-Detection/blob/master/HeadlineDetectionAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df=pd.read_excel("/content/drive/MyDrive/Research/NLP/Sentiment Data/FakevsAuthentic news edit sany.xlsx")

In [ ]:
df.head(5)

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
len(df[df.Label=='Fake'])

In [ ]:
len(df[df.Label=='Authentic'])

In [ ]:
df.Label = df.Label.replace({'Authentic': 1, 'Fake': 0})

In [ ]:
df.shape

In [ ]:
df['Headline'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna()

In [ ]:
for i in range(5):
    print("Serial:",i+1)
    print("Text:",df.Headline[i])
  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
df['Label'].plot(kind='hist')

In [ ]:
stas = ['Authentic', 'Fake']

data = [237,206]

fig = plt.figure(figsize =(10, 7))
plt.pie(data, labels = stas)


plt.show()

In [ ]:
df['length'] = df['Headline'].str.split().apply(len)
df.head(10)

In [ ]:
df.hist(column='length', by='Label', bins=50,figsize=(10,5))

In [ ]:
df.length.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, log_loss, cohen_kappa_score, roc_auc_score, roc_curve
from sklearn import metrics
%matplotlib inline

In [ ]:
import string 
def remove_punc(s):
    table = str.maketrans({key: None for key in string.punctuation})
    return s.translate(table)

In [ ]:
x = df['Headline'].fillna(' ')
y = df['Label']
print(x.head())
print(y.head())

In [ ]:
x = df['Headline'].apply(remove_punc)
x.shape

In [ ]:
contractions = { 
"বি.দ্র ": "বিশেষ দ্রষ্টব্য",
"ড.": "ডক্টর",
"ডা.": "ডাক্তার",
"ইঞ্জি:": "ইঞ্জিনিয়ার",
"রেজি:": "রেজিস্ট্রেশন",
"মি.": "মিস্টার",
"মু.": "মুহাম্মদ",
"মো.": "মোহাম্মদ",
}

In [ ]:
import re
import string
from collections import Counter
vocab = Counter()
def clean_text(x,remove_stopwords = True):
    if True:
        new_text = []
        for word in x:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        x = " ".join(new_text)
    # Format words and remove unwanted characters
    whitespace = re.compile(u"[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)
    bangla_digits = u"[\u09E6\u09E7\u09E8\u09E9\u09EA\u09EB\u09EC\u09ED\u09EE\u09EF]+"
    english_chars = u"[a-zA-Z0-9]"
    punc = u"[(),$%^&*+={}\[\]:\"|\'\~`<>/,¦!?½£¶¼©⅐⅑⅒⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞⅟↉¤¿º;-]+"
    bangla_fullstop = u"\u0964"     #bangla fullstop(dari)
    punctSeq   = u"['\"“”‘’]+|[.?!,…]+|[:;]+"
    
    x = re.sub(bangla_digits, " ", x)
    x = re.sub(punc, " ", x)
    x = re.sub(english_chars, " ", x)
    x = re.sub(bangla_fullstop, " ", x)
    x = re.sub(punctSeq, " ", x)
    x = whitespace.sub(" ", x).strip()
    
    x = re.sub(r'https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE)
    x = re.sub(r'\<a href', ' ', x)
    x = re.sub(r'&amp;‘:‘ ’', '', x) 
    x = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]। ,', ' ', x)
    x = re.sub(r'<br />', ' ', x)
    x = re.sub(r'\'', ' ', x)
    x = re.sub(r"[\@$#%~+-\.\'।\"]"," ",x)
    x = re.sub(r"(?m)^\s+", "", x)
    x = re.sub("[()]","",x)
    x = re.sub("[‘’]","",x)
    x = re.sub("[!]","",x)
    x = re.sub("[/]","",x)
    x = re.sub("[:]","",x)
    x = re.sub('\ |\?|\.|\!|\/|\;|\:', ' ',x)
    x = x.strip("/")
    
    if remove_stopwords:
        k = []
        with open('/content/drive/MyDrive/Research/NLP/Banglastopword.txt', 'r',encoding="utf-8") as f:
            for word in f:
                word = word.split()
                k.append(word[0])
            x = [t for t in x if t not in k]
            x = "".join(x)
   

    return x

In [ ]:
vocab = CountVectorizer(ngram_range=(1,1),analyzer='word',encoding='utf-8').fit(x)

In [ ]:
print(vocab.vocabulary_)

print("\n")

x = vocab.transform(x) 
print(x.toarray())

print("\n")

print(vocab.get_feature_names())

In [ ]:
print("Shape of the sparse matrix: ", x.shape)
print("Non-Zero occurences: ",x.nnz)
density = (x.nnz/(x.shape[0]*x.shape[1]))*100
print("Density of the matrix = ",density)

**Train Test & Split**

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=1000)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(x_train,y_train)
print('Confusion matrix of model is :')
cm = confusion_matrix(y_test,mnb.predict(x_test))
TP = cm[0][0]
TN = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]
print(cm)
print()
result1 = classification_report(y_test, mnb.predict(x_test))
print("Classification Report : ",)
print (result1)
print()
var1 = ((TP + TN)/(TP + TN + FP + FN)) *100
print('Testing accuracy : ',var1)
print('Sensitivity : ', TP/(TP+FN))
print('Specificity : ', TN/(TN+FP))
print('false positive rate : ', FP/(FP+TN))
print('false negative rate : ', FN/(FN+TP))
print('Negative Predictive Value : ', TN/(TN+FN))
print('False Discovery rate : ', FP/(TP+FP))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,mnb.predict(x_test)))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, mnb.predict(x_test)))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test,mnb.predict(x_test))))
print('Log_Loss:', metrics.log_loss(y_test,mnb.predict(x_test)))
print('Cohen_Kappa_Score:', cohen_kappa_score(y_test, mnb.predict(x_test)))

print()
print()
name = ['Multinomial Naive Bayes']
col_value = ['#95a5a6']
model_accuracy = pd.Series(data=[var1], index=[name[0]])
fig = plt.figure(figsize=(5,5))
width = 0.75 
model_accuracy.sort_values().plot.bar(alpha=0.8, color=[col_value[0]])
plt.xticks(rotation=0)
plt.title('Model Accracy')
plt.ylabel('Accuracy (%)')
plt.show()
print()
print()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(x_train,y_train)
print('Confusion matrix of model is :')
cm = confusion_matrix(y_test,rmfr.predict(x_test))
TP = cm[0][0]
TN = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]
print(cm)
print()
result1 = classification_report(y_test, rmfr.predict(x_test))
print("Classification Report : ",)
print (result1)
print()
var2 = ((TP + TN)/(TP + TN + FP + FN)) *100
print('Testing accuracy : ',var2)
print('Sensitivity : ', TP/(TP+FN))
print('Specificity : ', TN/(TN+FP))
print('false positive rate : ', FP/(FP+TN))
print('false negative rate : ', FN/(FN+TP))
print('Negative Predictive Value : ', TN/(TN+FN))
print('False Discovery rate : ', FP/(TP+FP))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,rmfr.predict(x_test)))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, rmfr.predict(x_test)))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test,rmfr.predict(x_test))))
print('Log_Loss:', metrics.log_loss(y_test,rmfr.predict(x_test)))
print('Cohen_Kappa_Score:', cohen_kappa_score(y_test, rmfr.predict(x_test)))

print()
print()
name = ['RandomForest']
col_value = ['red']
model_accuracy = pd.Series(data=[var2], index=[name[0]])
fig = plt.figure(figsize=(5,5))
width = 0.75 
model_accuracy.sort_values().plot.bar(alpha=0.8, color=[col_value[0]])
plt.xticks(rotation=0)
plt.title('Model Accracy')
plt.ylabel('Accuracy (%)')
plt.show()
print()
print()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)
print('Confusion matrix of model is :')
cm = confusion_matrix(y_test,dt.predict(x_test))
TP = cm[0][0]
TN = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]
print(cm)
print()
result1 = classification_report(y_test, dt.predict(x_test))
print("Classification Report : ",)
print (result1)
print()
var3 = ((TP + TN)/(TP + TN + FP + FN)) *100
print('Testing accuracy : ',var3)
print('Sensitivity : ', TP/(TP+FN))
print('Specificity : ', TN/(TN+FP))
print('false positive rate : ', FP/(FP+TN))
print('false negative rate : ', FN/(FN+TP))
print('Negative Predictive Value : ', TN/(TN+FN))
print('False Discovery rate : ', FP/(TP+FP))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,dt.predict(x_test)))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, dt.predict(x_test)))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test,dt.predict(x_test))))
print('Log_Loss:', metrics.log_loss(y_test,dt.predict(x_test)))
print('Cohen_Kappa_Score:', cohen_kappa_score(y_test, dt.predict(x_test)))

print()
print()
name = ['DecisionTree']
col_value = ['#9b59b6']
model_accuracy = pd.Series(data=[var3], index=[name[0]])
fig = plt.figure(figsize=(5,5))
width = 0.75 
model_accuracy.sort_values().plot.bar(alpha=0.8, color=[col_value[0]])
plt.xticks(rotation=0)
plt.title('Model Accracy')
plt.ylabel('Accuracy (%)')
plt.show()
print()
print()

In [ ]:
from sklearn.svm import SVC
import pickle
svm = SVC()
svm.fit(x_train,y_train)
print('Confusion matrix of model is :')
cm = confusion_matrix(y_test,svm.predict(x_test))
TP = cm[0][0]
TN = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]
print(cm)
print()
result1 = classification_report(y_test, svm.predict(x_test))
print("Classification Report : ",)
print (result1)
print()
var4 = ((TP + TN)/(TP + TN + FP + FN)) *100
print('Testing accuracy : ',var4)
print('Sensitivity : ', TP/(TP+FN))
print('Specificity : ', TN/(TN+FP))
print('false positive rate : ', FP/(FP+TN))
print('false negative rate : ', FN/(FN+TP))
print('Negative Predictive Value : ', TN/(TN+FN))
print('False Discovery rate : ', FP/(TP+FP))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,svm.predict(x_test)))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, svm.predict(x_test)))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test,svm.predict(x_test))))
print('Log_Loss:', metrics.log_loss(y_test,svm.predict(x_test)))
print('Cohen_Kappa_Score:', cohen_kappa_score(y_test, svm.predict(x_test)))

print()
print()
name = ['Support Vector Clasiffier']
col_value = ['blue']
model_accuracy = pd.Series(data=[var4], index=[name[0]])
fig = plt.figure(figsize=(5,5))
width = 0.75 
model_accuracy.sort_values().plot.bar(alpha=0.8, color=[col_value[0]])
plt.xticks(rotation=0)
plt.title('Model Accracy')
plt.ylabel('Accuracy (%)')
plt.show()
print()
print()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(x_train,y_train)
print('Confusion matrix of model is :')
cm = confusion_matrix(y_test,knn.predict(x_test))
TP = cm[0][0]
TN = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]
print(cm)
print()
result1 = classification_report(y_test, knn.predict(x_test))
print("Classification Report : ",)
print (result1)
print()
var5 = ((TP + TN)/(TP + TN + FP + FN)) *100
print('Testing accuracy : ',var5)
print('Sensitivity : ', TP/(TP+FN))
print('Specificity : ', TN/(TN+FP))
print('false positive rate : ', FP/(FP+TN))
print('false negative rate : ', FN/(FN+TP))
print('Negative Predictive Value : ', TN/(TN+FN))
print('False Discovery rate : ', FP/(TP+FP))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,knn.predict(x_test)))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, knn.predict(x_test)))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test,knn.predict(x_test))))
print('Log_Loss:', metrics.log_loss(y_test,knn.predict(x_test)))
print('Cohen_Kappa_Score:', cohen_kappa_score(y_test, knn.predict(x_test)))

print()
print()
name = ['K-NN']
col_value = ['#7fcdbb']
model_accuracy = pd.Series(data=[var5], index=[name[0]])
fig = plt.figure(figsize=(5,5))
width = 0.75 
model_accuracy.sort_values().plot.bar(alpha=0.8, color=[col_value[0]])
plt.xticks(rotation=0)
plt.title('Model Accracy')
plt.ylabel('Accuracy (%)')
plt.show()
print()
print()

In [ ]:
from sklearn.linear_model import LogisticRegression
l = LogisticRegression()
l.fit(x_train,y_train)
logi=l.score(x_test,  y_test)
print('Confusion matrix of model',i , 'is :')
cm = confusion_matrix(y_test,l.predict(x_test))
TP = cm[0][0]
TN = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]
print(cm)
print()
result1 = classification_report(y_test, l.predict(x_test))
print("Classification Report : ",)
print (result1)
print()
var6 = ((TP + TN)/(TP + TN + FP + FN)) *100
print('Testing accuracy : ',var6)
print('Sensitivity : ', TP/(TP+FN))
print('Specificity : ', TN/(TN+FP))
print('false positive rate : ', FP/(FP+TN))
print('false negative rate : ', FN/(FN+TP))
print('Negative Predictive Value : ', TN/(TN+FN))
print('False Discovery rate : ', FP/(TP+FP))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,l.predict(x_test)))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, l.predict(x_test)))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test,l.predict(x_test))))
print('Log_Loss:', metrics.log_loss(y_test,l.predict(x_test)))
print('Cohen_Kappa_Score:', cohen_kappa_score(y_test, l.predict(x_test)))

print()
print()
name = ['Logistic Regression']
col_value = ['#2ecc71']
model_accuracy = pd.Series(data=[var6], index=[name[0]])
fig = plt.figure(figsize=(5,5))
width = 0.75 
model_accuracy.sort_values().plot.bar(alpha=0.8, color=[col_value[0]])
plt.xticks(rotation=0)
plt.title('Model Accracy')
plt.ylabel('Accuracy (%)')
plt.show()
print()
print()

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
 
objects = ('MNB','RF','DT','SVC','KNN','logi')
y_pos = np.arange(len(objects))
performance = [var1,var2,var3,var4,var5,var6]
 
plt.bar(y_pos, performance, align='center', alpha=0.9)
plt.xticks(y_pos, objects)
plt.ylabel('Accuracy')
plt.title('Best Accuracy')

plt.show()

In [ ]:
# news=input()
# prt=vocab.transform([news])

# m1=mnb.predict(prt)
# print("MNV Predicted Rating:",m1)
# if m1==0:
#   print("No")
# else:
#   print("yes")

# m2=rmfr.predict(prt)
# print("RF Predicted Rating:",m2)
# if m2==0:
#   print("No")
# else: 
#   print("yes")


# m3=dt.predict(prt)
# print("DT Predicted Rating:",m3)
# if m3==0:
#   print("No")
# else:
#   print("yes")


# m4=svm.predict(prt)
# print("SVM Predicted Rating:",m4)
# if m4==0:
#   print("No")
# else:
#   print("Yes")

# m5=knn.predict(prt)
# print("KNN Predicted Rating:",m4)
# if m4==0:
#   print("No")
# else:
#   print("Yes")


#   m6=l.predict(prt)
# print("Logistic Predicted Rating:",m4)
# if m4==0:
#   print("No")
# else:
#   print("Yes")
